<a href="https://colab.research.google.com/github/nariba/hello-pycuda/blob/main/SourceModule.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install pycuda

     |████████████████████████████████| 1.6MB 2.8MB/s 
     |████████████████████████████████| 71kB 5.8MB/s 
     |████████████████████████████████| 81kB 6.6MB/s 
  Created wheel for pycuda: filename=pycuda-2020.1-cp36-cp36m-linux_x86_64.whl size=620917 sha256=4f42e036d3250abc61c601c1d1b9381358d28f1bbbb98d75eaeebf2c27a9799f
  Stored in directory: /root/.cache/pip/wheels/8f/78/d1/5bb826f81d9d490297a348d818ff3ee6dd6f2075b06dde6ea0
  Created wheel for pytools: filename=pytools-2020.4.3-py2.py3-none-any.whl size=61374 sha256=45428b3146dbb85225f1142e7712e984da12fcd9ecfabba797a222b1ace1d61e
  Stored in directory: /root/.cache/pip/wheels/af/c7/81/a22edb90b0b09a880468b2253bb1df8e9f503337ee15432c64
Successfully built pycuda pytools


In [2]:
import math
import numpy
import pycuda.gpuarray
from pycuda.compiler import SourceModule
import pycuda.autoinit

In [6]:
module = SourceModule("""
__global__ void add_two_array_2d(int nx, int ny, float *res, float *arr1, float *arr2){
  int x = threadIdx.x + blockDim.x * blockIdx.x;
  int y = threadIdx.y + blockDim.y * blockIdx.y;
  int ij = nx * y + x;
  if (x < nx && y < ny) {
    res[ij] = arr1[ij] + arr2[ij];
  }
}
""")

In [7]:
add_two_array = module.get_function("add_two_array_2d")

In [9]:
num_x, num_y = numpy.int32(5), numpy.int32(2)
num_components = num_x * num_y

In [10]:
arr1 = numpy.arange(num_components, dtype=numpy.float32).reshape(num_y, num_x)

In [12]:
numpy.random.seed(123)
arr2 = 10 * np.random.rand(num_y, num_x)
arr

NameError: ignored